# Tutorial: How to Select A-DBSCAN Parameters

[A-DBSCAN](https://pysal.org/esda/generated/esda.adbscan.ADBSCAN.html#esda.adbscan.ADBSCAN) is the clustering algorithm used in `stco`. There are two primary parameters that are needed to run A-DBSCAN, regardless of whether it is run directly from `esda` or through the `TemporalADBSCAN` in `stco`, which are *eps* and *min_samples*.

In this notebook, the process of selecting these parameters is walked through in a detailed manner.

## Eps
Eps can be defined as, ["the maximum distance between two samples for them to be considered as in the same neighborhood"](https://pysal.org/esda/generated/esda.adbscan.ADBSCAN.html#esda.adbscan.ADBSCAN). This allows us to define how close points must be to be clustered together.

One important note is that this value is in the same units as the coordinate system of the input data. So for example, if the data are points that use lat/long, then the unit of eps is degrees.

For example, see below where 0.15 is the eps and means that we are looking at 0.15 degrees, which roughly tranlates to about 17 km. As a general rule of thumb, 1 degree is about 111 km, but this varies widely since the Earth is distorted, especially closer to the poles.

```python
tadbs = TemporalADBSCAN(data=my_dataframe, period="Y", eps=0.15, min_sample_pct=0.01)
```

### Changing the Units

Sometimes we want to be a little more precise with our measurements, rather than just guessing. Luckily, with GeoPandas we are able to easily transform the coordinate system of the dataset to get it into a better system that uses our preferred units. It is important to carefully think out what coordinate system you should use, because not all coordinate systems are intended for use anywhere.

In the example of Minnesota accident points, where the input data is represented as lat/long (using the WGS84 Coordinate System, EPSG:4326), a couple reasonable coordinate systems to use might be NAD83 / UTM zone 15N (EPSG:26915) or WGS 84 / Pseudo-Mercator (EPSG:3857). Both of these options use meters as the unit, while the input of WGS84 uses degrees.

For more information on available coordinate systems, check out [epsg.io](https://epsg.io/). For more information on how coordinate systems work, check out Esri's documentation [here](https://pro.arcgis.com/en/pro-app/3.1/help/mapping/properties/coordinate-systems-and-projections.htm).

In [1]:
# Import
import geopandas as gpd

# Read in data
incidents = gpd.read_file("mn_accidents.geojson")

# Check current spatial reference
incidents.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [2]:
# Change spatial reference
incidents = incidents.to_crs(epsg=3857)

# Check spatial reference again
incidents.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [3]:
# Look at data - geometry no longer using degrees!
# Now we can use meters for our eps unit in ADBSCAN/TemporalADBSCAN
incidents

,icr,incident_type,incident_date,district,location_description,road_condition,vehicles_involved,city_id,geometry
0,20601015,Injury,2020-03-04 07:35:00,2600 St. Cloud,"Highway 4 at County Road 28, Danielson Twp, Me...",Dry,2,2,POINT (-10541569.500 5627356.138)
1,17901947,Injury,2017-09-09 18:39:00,2900 Detroit Lakes,"Eastbound Hwy 10 between Bluffton and Wadena, ...",Dry,1,3,POINT (-10601375.156 5855512.170)
2,20902058,Injury,2020-12-07 16:30:00,2900 Detroit Lakes,"Westbound Hwy 10 MP87, BLUFFTON TWP, Otter Tai...",Dry,1,3,POINT (-10602609.110 5855651.868)
3,19901575,Injury,2019-07-28 13:28:00,2900 Detroit Lakes,"10 Highway / 77 County Road, Bluffton, Otter T...",Wet,1,3,POINT (-10601465.708 5855682.091)
4,17900227,Injury,2017-01-21 10:58:00,2900 Detroit Lakes,"East bound Hwy 10 west of Bluffton, Bluffton T...",Wet,3,3,POINT (-10602609.110 5855651.868)
...,...,...,...,...,...,...,...,...,...
9954,17104577,Injury,2017-10-14 10:14:00,2100 Rochester,"Highway 63 at Interstate 90, High Forest Twp, ...",Wet,2,2744,POINT (-10295746.011 5447658.245)
9955,22102611,Injury,2022-05-30 14:29:00,2100 Rochester,"Southbound Highway 63 at westbound I-90 ramp, ...",Dry,1,2744,POINT (-10295746.011 5447658.245)
9956,22103710,Injury,2022-08-08 13:41:00,2100 Rochester,"SB HWY 63 at WB I-90, HIGH FOREST TWP, Olmsted...",Dry,2,2744,POINT (-10295746.011 5447658.245)
9957,17100813,Injury,2017-02-14 11:00:00,2100 Rochester,"Hwy 63 and 6th Street Stewartville, High Fores...",Dry,2,2744,POINT (-10295771.615 5442002.370)


## Min Samples

Min samples can be defined as, ["the number of samples (or total weight) in a neighborhood for a point to be considered as a core point"](https://pysal.org/esda/generated/esda.adbscan.ADBSCAN.html#esda.adbscan.ADBSCAN). It allows us to define how many points are needed to be considered a cluster.

An important distinction to make here is that `ADBSCAN` in `esda` uses an integer to represent the actual number of minimum samples to use. `TemporalADBSCAN` in `stco` uses a float to represent a given **percentage** of the total number of points to use as the number of minimum samples, **for each individual period of time**.

So in other words, if you were to pass in 0.01 as the `min_samples` parameter for `TemporalADBSCAN`, then for each period of time (e.g., year, month, week, whatever you choose), 1% of the total number of features for that period will be used as the number of features. For example, if there were 1000 features in time step 1, then `1000 * 0.01 = 10`. Then if time step 2 had 1500 features, `1500 * 0.01 = 15`. So each period would have the same percentage used, but the raw values would differ.

### Finding an Appropriate Percentage

The first step at figuring out what percentage to use is to actually break the data down and see how many values there are in each period. Below, we will look at how many accidents there are for each year.

An important thing to note is that the period type used below (denoted as a single letter like "Y" for Year, "W" for week, etc...) should be the same value that will get passed into TemporalADBSCAN's period parameter.

In [4]:
# Choose period value
period_value = "Y"

# Create new column with period for each record
incidents["PERIOD"] = incidents["incident_date"].dt.to_period(period_value).astype(str)

# Display data
incidents.head()

,icr,incident_type,incident_date,district,location_description,road_condition,vehicles_involved,city_id,geometry,PERIOD
0,20601015,Injury,2020-03-04 07:35:00,2600 St. Cloud,"Highway 4 at County Road 28, Danielson Twp, Me...",Dry,2,2,POINT (-10541569.500 5627356.138),2020
1,17901947,Injury,2017-09-09 18:39:00,2900 Detroit Lakes,"Eastbound Hwy 10 between Bluffton and Wadena, ...",Dry,1,3,POINT (-10601375.156 5855512.170),2017
2,20902058,Injury,2020-12-07 16:30:00,2900 Detroit Lakes,"Westbound Hwy 10 MP87, BLUFFTON TWP, Otter Tai...",Dry,1,3,POINT (-10602609.110 5855651.868),2020
3,19901575,Injury,2019-07-28 13:28:00,2900 Detroit Lakes,"10 Highway / 77 County Road, Bluffton, Otter T...",Wet,1,3,POINT (-10601465.708 5855682.091),2019
4,17900227,Injury,2017-01-21 10:58:00,2900 Detroit Lakes,"East bound Hwy 10 west of Bluffton, Bluffton T...",Wet,3,3,POINT (-10602609.110 5855651.868),2017


In [5]:
# Group data by the period and find the total number of features for each
incidents.groupby(["PERIOD"])["PERIOD"].count()

PERIOD
2017    1707
2018    1640
2019    1586
2020    1273
2021    1384
2022    1256
2023    1113
Name: PERIOD, dtype: int64

In [6]:
# We can also look at the mean number for each period
incidents.groupby(["PERIOD"])["PERIOD"].count().mean()

1422.7142857142858

This means that on average, if 0.01 (1%) was used as the `min_samples_percentage` parameter for TemporalADBSCAN, then 14 or so would the the average `min_samples` parameter used by A-DBSCAN for each period.

This seems like a reasonable number! But we could adjust the percentage to get to our desired average value.

## The Relationship between Eps and Min Samples

There is also another way to determine the ideal values for the parameters, by looking at the relationship between the two parameters. As noted in the `esda` [tutorial](https://pysal.org/esda/notebooks/adbscan_berlin_example.html) on A-DBSCAN, we can use the desried density in conjunction with one of the parameters to calculate the other. Mathematically, this is defined as:

$$Density = {Min \text{ } Samples \over \pi * Eps^2} $$

So in our example above, where we decided on using 1% for the minimum sample percentage (on average, a value of 142) and 0.15 degrees for the eps (roughly 17 km), we can calculate the density at which we consider a cluster as:

$$Density = {14 \text{ } accidents \over \pi * 17 km^2} \text{ or } 0.015 \text{ } accidents / km^2 $$

### What if we want to change the density?

Maybe we want to achieve an accident density of 0.025 instead.

We can do this two ways, either by keeping the eps at 0.15 degrees (17 km) or by keeping the min sample percentage at 1% (14 accidents). We will do it both ways below.

#### Eps = 0.15 degrees (17km) & Unknown Min Samples

$$Min\text{ }Samples = {0.025 \text{ } accidents / km^2 * \pi * 17 km^2} \text{ or } 23 \text{ } accidents \text{ }(~1.6 \text{ } percent)$$

Output: 1.6% min sample percentage, 17km eps

#### Min Samples = 1% (14) & Unknown Eps

$$Eps = \sqrt{14 \text{ } accidents \over \pi * 0.025 \text{ } accidents / km^2} \text{ or } 13.4 \text{ } km \text{ }(~0.12 \text{ } degrees)$$

Output: 1% min sample percentage, 13km eps